ニューラルネットワークで学習してみるファイル

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

import utils

# For reproducibility
np.random.seed(42)
torch.manual_seed(42) # 乱数生成シード
cudnn.benchmark = True

# Grab a GPU if there is one
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using {} device: {}".format(device, torch.cuda.current_device()))
else:
    device = torch.device("cpu")
    print("Using {}".format(device))

Using cuda device: 0


Load dataset

In [2]:
fold = "../train_raw_npy/"
xyz = np.load(f"{fold}acc_xyz.npy")
label = np.load(f"{fold}sampled_label.npy")
print(xyz.shape)
print(label.shape)


(196072, 3, 500)
(196072,)


train / test spilit

In [3]:
pin = 170000
train_xyz = xyz[0:pin,:,:]
test_xyz = xyz[pin:190000, :, :]
train_label = label[0:pin]
test_label = label[pin:190000]
print(train_xyz.shape)
print(test_xyz.shape)

(170000, 3, 500)
(20000, 3, 500)


In [4]:
print(pd.Series(train_label).value_counts())
print(pd.Series(test_label).value_counts())

7.0    27641
5.0    26056
6.0    25689
8.0    22016
2.0    21185
1.0    20712
4.0    19603
3.0     7098
Name: count, dtype: int64
7.0    3612
1.0    3315
4.0    2766
6.0    2637
2.0    2489
5.0    2419
8.0    1818
3.0     944
Name: count, dtype: int64


In [5]:
# trainをランダムに20000個サンプリングする。
sample_size = 20000
indices = np.random.choice(train_xyz.shape[0], sample_size, replace=False)
train_xyz_random = train_xyz[indices]
train_label_random = train_label[indices]


In [6]:
print(train_xyz_random.shape)

(20000, 3, 500)


# アーキテクチャデザイン  
オックスフォードでの例にあるようにCNNを使う

In [7]:
class ConvBNReLU(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,

            kernel_size=3,
            stride=1,
            padding=1,
            bias=True
    ):
        super(ConvBNReLU, self).__init__()

        self.main = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=bias),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(True)
        )
    
    def forward(self,x):
        return self.main(x)


In [ ]:
class CNN(nn.Module):
    def __init__(self, output_size=8, in_channels=3, num_filters_init=8):
        super(CNN, self).__init__()

        self.cnn = nn.Sequential(
            ConvBNReLU(in_channels, num_filters_init, 8,4,3) # 500 -> 250 
            ConvBNReLU(num_filters_init, num_filters_init*2, ) # 250 -> 125 (248)
        )